In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date
from datetime import datetime, date
from dateutil.rrule import rrule, MONTHLY, DAILY, WEEKLY
from dateutil.relativedelta import relativedelta
from dateutil.parser import parse
from word2number import w2n
import re
import calendar

In [2]:
postdate = "3/7/2018  12:00:00 AM"

In [3]:
subtract = "(before|earlier|ago|back|last|past|previous)"
add = "(after|later|next|coming|this)"
numbers = "(^a(?=\s)|one|two|three|four|five|six|seven|eight|nine|ten| eleven|twelve|thirteen|fourteen|fifteen|sixteen|seventeen|eighteen|nineteen|twenty|thirty|forty|fifty|sixty|seventy|eighty|ninety|hundred|thousand)"
dmy = "(year|day|week|month)"
month = "(january|february|march|april|may|june|july|august|september|october|november|december|jan|feb|aug|sep|oct|nov|dec)"
years_subtract = "(last|previous)"
years_add = "(next)"
years_noChange = "(coming|this)"
year = "((?<=\s)\d{4}|^\d{4})"
iso = "\d+[/-]\d+"

In [4]:
subtractexp = re.compile(subtract, re.IGNORECASE)
addexp = re.compile(add, re.IGNORECASE)
numbexp = re.compile(numbers, re.IGNORECASE)
dmyexp = re.compile(dmy, re.IGNORECASE)
monthexp = re.compile(month, re.IGNORECASE)
yearsAddexp = re.compile(years_add, re.IGNORECASE)
yearsSubtractexp = re.compile(years_subtract, re.IGNORECASE)
yearsNoChangeexp = re.compile(years_noChange, re.IGNORECASE)
yearexp = re.compile(year)
isoexp = re.compile(iso)

In [5]:
def relative_change(relative, digit=1):
    if relative=='day':
        return relativedelta(days=digit)
    if relative=='month':
        return relativedelta(months=digit)
    if relative=='week':
        return relativedelta(weeks=digit)
    if relative=='year':
        return relativedelta(years=digit)

In [6]:
def monthToNum(Month):
    return{
            'jan' : 1,
            'feb' : 2,
            'mar' : 3,
            'apr' : 4,
            'may' : 5,
            'jun' : 6,
            'jul' : 7,
            'aug' : 8,
            'sep' : 9, 
            'oct' : 10,
            'nov' : 11,
            'dec' : 12,
            'january' : 1,
            'february' : 2,
            'march' : 3,
            'april' : 4,
            'june' : 6,
            'july' : 7,
            'august' : 8,
            'september' : 9, 
            'october' : 10,
            'november' : 11,
            'december' : 12
    }[Month]

In [7]:
# Find numbers
def temp_to_data(temp,postdate):
    try:
        digits_present = re.findall(r'\b\d+\b', temp)
        wordnum_present = numbexp.findall(temp)
        dmy_present = dmyexp.findall(temp)
        year_present = yearexp.findall(temp)
        month_present = monthexp.findall(temp)
        iso_present = isoexp.findall(temp)
        actual_date = parse(postdate)

        ## 1 month/year/day ago/after
        if (len(digits_present)>0):
            print "Case 1"
            digit = digits_present[0]
            dmy_present = dmyexp.findall(temp)
            if len(dmy_present)>0:
                dmy = dmy_present[0]
    #             actual_date = parse(postdate)
                if(len(subtractexp.findall(temp))>0):
                    x= actual_date-relative_change(dmy.lower(), int(digit))
                    return x
                elif(len(addexp.findall(temp))>0):
                    x= actual_date+relative_change(dmy.lower(), int(digit))
                    return x

        # One month/day/week ago/after
        elif (len(wordnum_present)>0):
            print "Case 2"
            words = wordnum_present[0]
            digit = w2n.word_to_num(words)
            dmy_present = dmyexp.findall(temp)
            if len(dmy_present)>0:
                dmy = dmy_present[0]
    #             actual_date = parse(postdate)
                if(len(subtractexp.findall(temp))>0):
                    x= actual_date-relative_change(dmy.lower(), int(digit))
                    return x
                elif(len(addexp.findall(temp))>0):
                    x= actual_date+relative_change(dmy.lower(), int(digit))
                    return x

        #Last month/day/week
        elif len(dmy_present)>0:
            print "Case 3"
            dmy = dmy_present[0]
    #         actual_date = parse(postdate)
            if(len(subtractexp.findall(temp))>0):
                x= actual_date-relative_change(dmy.lower())
                return x
            elif(len(addexp.findall(temp))>0):
                x= actual_date+relative_change(dmy.lower())
                return x

        # last/since/next/this Nov/November    
        elif len(month_present)>0:
            print "Case 4"
            month_name = month_present[0]
            month_number = monthToNum(month_name.lower())
    #         actual_date = parse(postdate)
            if(len(yearsSubtractexp.findall(temp))>0):
                x= actual_date-relativedelta(years=1,month=int(month_number))
                return x
            elif(len(yearsAddexp.findall(temp))>0):
                x= actual_date+relativedelta(years=1,month=int(month_number))
                return x
            elif(len(re.findall("since", temp, re.IGNORECASE))>0):
                diff = actual_date.month - month_number
                if(diff>0):
                    return actual_date+relativedelta(month=int(month_number))
                elif(diff<0):
                    x= actual_date-relativedelta(years=1,month=int(month_number))
                    return x
            else:
                x = actual_date+relativedelta(month=month_number) 
                return x
            
        #today
        elif (len(re.findall("today", temp, re.IGNORECASE))>0):
            x = actual_date
        
        #tomorrow 
        elif (len(re.findall("(tomorrow|tom)", temp, re.IGNORECASE))>0):
            x = actual_date + relativedelta(days=1)
        
        #yesterday
        elif (len(re.findall("(yesterday)", temp, re.IGNORECASE))>0):
            x = actual_date - relativedelta(days=1)

        else:
            print temp

        # Nov/November 2011
        if (len(year_present)>0 and len(month_present)>0):
            print "Case 5"
            month_name = month_present[0]
            month_number = monthToNum(month_name.lower())
    #         actual_date = parse(postdate)
            x = actual_date+relativedelta(year=int(year_present[0]),month=int(month_number),day=15)

            return x
        
        # 2011
        if len(year_present)>0:
            x = actual_date+relativedelta(year=int(year_present[0]),month=1,day=15)
            
        # iso
        if len(iso_present)>0:
            val = re.split("(/|-)",iso_present[0])
            if(len(val[-1])!=4):
                year = '20'+val[-1]
            else:
                year = val[-1]
            x = actual_date+relativedelta(year=int(year),month=val[0],day=15)
            
    except:
        print temp

In [8]:
selected_users= pd.read_csv('Seletec_users_100_tagged.csv')

In [9]:
def call_func(temp,postdate):
    splitted = temp.strip("[]").rsplit(",")
    data = []
    print type(splitted)
    if (len(splitted)==0):
        return []
    else:
        for split in splitted:
            data.append(temp_to_data(split,postdate))
        return data

In [17]:
for row in selected_users.itertuples():
    print row
    print row['temp']
#     selected_users["temp2date"] = call_func(row["temp"], row["date"])

Pandas(Index=0, _1=0, id=4782, _3='The plan is a suprficial node dissection (open extraction) on positive side w/ high dose Interferon infusion for 30 days and 3 day per week injection to follow.', date='4/16/2011 0:00', forum='1', User='awg', Tag='1', User_id=38, Sent_list=8, sentence_id='4782_8', _11=nan, temp="['3 day' '30 days']", Drug="['Interferon']", Negations='[]')


TypeError: tuple indices must be integers, not str

In [ ]:
selected_users.to_csv("Selected_100Users_temp2date.csv",index=False)

In [ ]:
selected_users.head(5)